In [1]:
import seamless
from seamless.highlevel import Context, Transformer, Cell

seamless.delegate(False)
ctx = Context()

In [2]:
ctx.pdb1 = open("1AKE-flanks.pdb").read()
ctx.pdb2 = open("1AKE-B-hit.pdb").read()
ctx.load_pdb1 = Transformer()
ctx.load_pdb1.pdb = ctx.pdb1
ctx.load_pdb_code = ctx.load_pdb1.code.pull()
ctx.load_pdb_code.mount("load_pdb.py")
ctx.coor1 = ctx.load_pdb1
ctx.load_pdb2 = Transformer()
ctx.load_pdb2.pdb = ctx.pdb2
ctx.load_pdb2.code = ctx.load_pdb_code
ctx.coor2 = ctx.load_pdb2

In [3]:
await ctx.computation()
print(ctx.load_pdb1.logs)

Traceback (most recent call last):
  File "Seamless transformer: .load_pdb1.tf", line 2, in <module>
    import Bio.PDB
ModuleNotFoundError: No module named 'Bio'
*************************************************
Execution time: 0.0 seconds



In [4]:
environment_yml = """
channels:
- bioconda
- conda-forge
dependencies:
- biopython
"""
ctx.load_pdb1.environment.set_conda(environment_yml, "yaml")
ctx.load_pdb2.environment.set_conda(environment_yml, "yaml")

In [5]:
await ctx.computation()
print(ctx.load_pdb1.logs)

Traceback (most recent call last):
  File "Seamless transformer: .load_pdb1.tf", line 2, in <module>
    import Bio.PDB
ModuleNotFoundError: No module named 'Bio'
*************************************************
Execution time: 0.0 seconds



In [6]:
%pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
ctx.load_pdb1.clear_exception()
ctx.load_pdb2.clear_exception()
await ctx.computation()
print(ctx.load_pdb1.logs)

*************************************************
* Result
*************************************************
<binary buffer of length 320, checksum fe4621ada9594853b8c9c13e6fd0b63681df0e8b209079f57b2b74a220f58a94>
*************************************************
Execution time: 0.0 seconds
*************************************************


In [8]:
def bcscore(coor1, coor2, flanksize, gapsize):
    print("BCSCORE", flanksize, gapsize)
    return 0.5

ctx.bcscore = bcscore
ctx.bcscore.coor1 = ctx.coor1
ctx.bcscore.coor2 = ctx.coor2
ctx.bcscore.flanksize = 4
ctx.bcscore.gapsize = 7

ctx.bc_result = ctx.bcscore
await ctx.computation()
print(ctx.bc_result.value)

<Silk: 0.5 >


In [9]:
ctx.bcscore.language = "c"

ctx.bcscore_schema = Cell()
ctx.bcscore_schema.celltype = "plain"
ctx.bcscore_schema.mount("bcscore-schema.json", "r", authority="file")
await ctx.translation()
ctx.bcscore.result.example.set(1.0)
ctx.bcscore.result.schema["form"] = {"bytesize": 8}
await ctx.translation()
ctx.link(ctx.bcscore_schema, ctx.bcscore.inp.schema)
ctx.header = ctx.bcscore.header
ctx.header.mimetype = "h"
await ctx.translation()

In [10]:
ctx.bcscore.header.mount("bcscore.h", "w")
ctx.header.output()

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '\n/*\nThe following C header has been …

In [11]:
ctx.bcscore.code.mount("bcscore.c", authority="file")
await ctx.translation()
ctx.bcscore.result.example.set(1.0)
ctx.bcscore.result.schema["form"] = {"bytesize": 4}
ctx.bcscore.main_module.bclib.language = "c"
ctx.bclib_code = ctx.bcscore.main_module.bclib.code.pull()
ctx.bclib_code.mount("BClib.c")
await ctx.translation()



gcc -c -O3 -ffast-math -march=native -fPIC -fopenmp -p -g -o /tmp/seamless-extensions/seamless_module_d0712c61459d8273e9ab1318df172179b617a79e811da993a3110d5b8f9cd4ed_module/bclib.o /tmp/seamless-extensions/seamless_module_d0712c61459d8273e9ab1318df172179b617a79e811da993a3110d5b8f9cd4ed_module/bclib.c

gcc -c -O3 -ffast-math -march=native -fPIC -fopenmp -p -g -o /tmp/seamless-extensions/seamless_module_d0712c61459d8273e9ab1318df172179b617a79e811da993a3110d5b8f9cd4ed_module/main.o /tmp/seamless-extensions/seamless_module_d0712c61459d8273e9ab1318df172179b617a79e811da993a3110d5b8f9cd4ed_module/main.c



gcc -c -O3 -ffast-math -march=native -fPIC -fopenmp -p -g -o /tmp/seamless-extensions/seamless_module_bf6e993e786c8b9bad7d7fd8caec721be7048713c589f221bb6b1cc89e920420_module/bclib.o /tmp/seamless-extensions/seamless_module_bf6e993e786c8b9bad7d7fd8caec721be7048713c589f221bb6b1cc89e920420_module/bclib.c

gcc -c -O3 -ffast-math -march=native -fPIC -fopenmp -p -g -o /tmp/seamless-extensions/s

In [12]:
await ctx.computation()
print(ctx.bcscore.inp.exception)
print(ctx.bcscore.exception)

None
None


In [13]:
import numpy as np

def validate_shape(self):
    assert self.coor1.shape[0] == 2 * self.flanksize, self.coor1.shape
    assert self.coor2.shape[0] == 2 * self.flanksize + self.gapsize, self.coor2.shape

example = ctx.bcscore.example
example.coor1 = np.zeros((10, 3))
example.coor2 = np.zeros((14, 3))
example.flanksize = 5
example.gapsize = 4
example.add_validator(validate_shape, name="validate_shape")

In [14]:
await ctx.computation()
print(ctx.bcscore.schema)
print(ctx.bcscore.result.schema)

{'properties': {'coor1': {'form': {'contiguous': True, 'ndim': 2, 'shape': [-1, 3]}, 'items': {'form': {'bytesize': 8, 'type': 'number'}}, 'storage': 'binary', 'type': 'array'}, 'coor2': {'form': {'contiguous': True, 'ndim': 2, 'shape': [-1, 3]}, 'items': {'form': {'bytesize': 8, 'type': 'number'}}, 'storage': 'binary', 'type': 'array'}, 'flanksize': {'type': 'integer'}, 'gapsize': {'type': 'integer'}}, 'type': 'object', 'validators': [{'code': 'def validate_shape(self):\n    assert self.coor1.shape[0] == 2 * self.flanksize, self.coor1.shape\n    assert self.coor2.shape[0] == 2 * self.flanksize + self.gapsize, self.coor2.shape\n', 'language': 'python', 'name': 'validate_shape'}]}
{'form': {'bytesize': 4}, 'type': 'number'}


In [15]:
await ctx.computation()
print(ctx.bc_result.value)

<Silk: 0.9981744289398193 >


In [16]:
ctx.save_graph("bcscore.seamless")
ctx.save_zip("bcscore.zip")